<a href="https://colab.research.google.com/github/antnolang/CronuSwim/blob/master/training/ML_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pre-procesado de datos

Imports necesarios, variables globales y montando drive para acceder a los datos de entrenamiento. Los datos de entrenamiento (eventX.csv) se encuentran en [esta carpeta de google drive](https://drive.google.com/drive/folders/1cJVks6-vDZHNLh_PZGftixH3rgFID53Q?usp=sharing).

In [ ]:
! pip install emlearn
import emlearn
import math
import glob
import os
import random
import pandas as pd
import scipy.stats as sp
import numpy as np
from google.colab import drive
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFECV
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics

random.seed(0)
drive.mount('drive/', force_remount=True)

W_SIZE = 70
FILE_PREFIX = '/content/drive/MyDrive/US/4º_(III)/TFG/data/'
TESTING_PATH = FILE_PREFIX + 'test/'
FILENAME_TARGET = FILE_PREFIX + 'swimming_phases.csv'
CLASSES_MAP_INT = {'S':0, 'O':1, 'F':2}

Mounted at drive/


Definiciones de las funciones de procesado de datos. Para equilibrar los datos de entrenamiento, se generará en el fichero de salida el mismo número de registros de START, FINISH y OTHERS. Los registros de OTHERS corresponden a uno de los registros de cada event elegido de forma aleatoria para añadir variabilidad a este tipo de movimiento.

In [ ]:
# Encuentra los signos '+' en el archivo indicado y comprueba que hay 
# exactamente 2. Importante que el archivo mantenga su cabecera
def find_plus_and_count(file):
    file.readline() # skip csv header
    plus_positions = []
    
    for line_num, line in enumerate(file):
        if '+' in line:
            plus_positions.append(line_num)

    if len(plus_positions) == 2:
        return tuple(plus_positions), line_num + 1
    else:
        raise Exception("ERROR: Incorrect format file.")


def get_statistic_from_sensor(x_axis, y_axis, z_axis, event_df, section):
    subset_data = event_df[[x_axis,y_axis,z_axis]][section[0]:section[1]]

    mean_x, mean_y, mean_z = subset_data.mean()
    median_x, median_y, median_z = subset_data.median()
    stdev_x, stdev_y, stdev_z = subset_data.std(ddof=0)
    var_x, var_y, var_z = subset_data.var(ddof=0)
    min_x, min_y, min_z = subset_data.min()
    max_x, max_y, max_z = subset_data.max()    
    kurtosis_x, kurtosis_y, kurtosis_z = sp.kurtosis(subset_data)
    skew_x, skew_y, skew_z = sp.skew(subset_data)
    q1_x, q1_y, q1_z = subset_data.quantile(0.25)
    q3_x, q3_y, q3_z = subset_data.quantile(0.75)
    iqr_x = q3_x - q1_x
    iqr_y = q3_y - q1_y
    iqr_z = q3_z - q1_z
    rms = math.sqrt(np.square(subset_data.to_numpy()).sum() / (W_SIZE*3))

    return [mean_x, mean_y, mean_z, median_x, median_y, median_z, 
            stdev_x, stdev_y, stdev_z, var_x, var_y, var_z, 
            min_x, min_y, min_z, max_x, max_y, max_z, 
            kurtosis_x, kurtosis_y, kurtosis_z, skew_x, skew_y, skew_z, 
            q1_x, q1_y, q1_z, q3_x, q3_y, q3_z, iqr_x, iqr_y, iqr_z, rms]


def process_window(section, filename_src, w_class):
    event_df = pd.read_csv(filename_src)
    accel_stats = get_statistic_from_sensor('aX', 'aY', 'aZ', event_df, section)
    gyros_stats = get_statistic_from_sensor('gX', 'gY', 'gZ', event_df, section)
    stats = accel_stats + gyros_stats
    str_stats = ','.join(str(s) for s in stats)

    write_header = not os.path.exists(FILENAME_TARGET)

    with open(FILENAME_TARGET, 'a') as file_target:
        if write_header: 
            header = ('mean_aX,mean_aY,mean_aZ,median_aX,median_aY,median_aZ,'
                  'stdev_aX,stdev_aY,stdev_aZ,var_aX,var_aY,var_aZ,'
                  'min_aX,min_aY,min_aZ,max_aX,max_aY,max_aZ,'
                  'kurtosis_aX,kurtosis_aY,kurtosis_aZ,skew_aX,skew_aY,skew_aZ,'
                  'q1_aX,q1_aY,q1_aZ,q3_aX,q3_aY,q3_aZ,iqr_aX,iqr_aY,iqr_aZ,'
                  'rms_a,mean_gX,mean_gY,mean_gZ,median_gX,median_gY,median_gZ,'
                  'stdev_gX,stdev_gY,stdev_gZ,var_gX,var_gY,var_gZ,'
                  'min_gX,min_gY,min_gZ,max_gX,max_gY,max_gZ,'
                  'kurtosis_gX,kurtosis_gY,kurtosis_gZ,skew_gX,skew_gY,skew_gZ,'
                  'q1_gX,q1_gY,q1_gZ,q3_gX,q3_gY,q3_gZ,iqr_gX,iqr_gY,iqr_gZ,'
                  'rms_g,class\n')
            file_target.write(header)

        file_target.write(str_stats + ',' + w_class + '\n')


def process_windows(sections, filename_src):
    process_window(sections[0], filename_src, 'S')
    process_window(sections[1], filename_src, 'F')
    process_window(random.choice(sections[2:]), filename_src, 'O')


# Devuelve tuplas en el siguiente orden: start, finish, other1, other2, other3, ... 
# Todos los rangos [init_line, last_line)
def get_sections(filename):
    file = open(filename, "r")
    plus_positions, line_counter = find_plus_and_count(file)
    others_count = 0
    result = []

    # Start section
    start_section_init = plus_positions[0] - math.floor(W_SIZE / 2)
    if start_section_init < 0:
        start_section_init = 0
    start_section_end = start_section_init + W_SIZE
    result.append((start_section_init, start_section_end))

    # Finish section
    finish_section_end = plus_positions[1] + math.floor(W_SIZE / 2)
    if finish_section_end + 1 > line_counter:
        finish_section_end = line_counter - 1
    finish_section_init = finish_section_end - W_SIZE
    result.append((finish_section_init, finish_section_end))

    # Other sections between Start and Finish sections
    start_finish_gap = finish_section_init - start_section_end
    if start_finish_gap < 0:
        raise Exception("ERROR: Collapsing start section and finish section.")
    elif start_finish_gap >= W_SIZE:
        other_section_number = math.floor(start_finish_gap / W_SIZE)
        other_section_init = start_section_end
        for _ in range(other_section_number):
            other_section_end = other_section_init + W_SIZE
            result.append((other_section_init, other_section_end))
            other_section_init = other_section_end

    # Other section before Start section
    before_start_gap = start_section_init - 0
    if before_start_gap > W_SIZE:
        other_static_section_number = round(other_section_number * 0.35)
        for _ in range(other_static_section_number):
            result.append((0, W_SIZE))
            
    return result


# =============================== Main function =============================== #

def process_data():
    if os.path.exists(FILENAME_TARGET): os.remove(FILENAME_TARGET)

    for filename in glob.glob(FILE_PREFIX + "event*.csv"):
        sections = get_sections(filename)
        process_windows(sections, filename)

    print("Window size: " + str(W_SIZE))

Llamada a la función principal de procesado de datos.

In [ ]:
process_data()

Window size: 70


Definimos una función para facilitar la lectura de los datos procesados. El modelo en C++ devuelve las clases en valores int. Por eso es necesario mapear las clases de string a enteros al entrenar el modelo y así nos aseguramos que el número 0 pertenece a la salida, el 2 a la llegada y el 1 al resto de movimientos.

Esta función se irá usando a lo largo de este documento:

In [ ]:
def read_X_y():
    df = pd.read_csv(FILENAME_TARGET)
    y = df['class']
    y = y.apply(lambda x: CLASSES_MAP_INT[x])
    X = df.drop(['class'], axis=1)
    return X, y

# Selección de las features del modelo

Ya que estamos trabajando en un software de tiempo real, el tiempo que se tarda en calcular las features es importante para que el dispositivo esté el mayor tiempo posible leyendo datos de entrada y no se pierda demasiada información. Para ello, es conveniente reducir el número de features a calcular.

Por otro lado cuanto mayor sea el número de features a usar, mejor precisión suele tener el algoritmo de machine learning. Por lo tanto hay que encontrar un buen equilibrio para que no haga falta calcular muchas features pero que al mismo tiempo den una buena precisión.

De las *N* posibles features a usar con el algoritmo de machine learning, podemos seleccionar las que queramos y el número que queramos. Por ejemplo, podemos seleccionar solo la feature *mean_aX*, o por ejemplo las features *median_aY*, *stdev_gX* y *kurtosis_aZ*, etc.

Por lo tanto, el número de combinaciones posibles de features para usar en el algoritmo de machine learning se puede calcular de la siguiente manera:

$$ \sum_{i=1}^{N} C_{N}^{i} = \sum_{i=1}^{N} \frac{N!}{i!(N-i)!} $$

In [ ]:
X, y = read_X_y()

N = X.columns.size - 1
print("N = " + str(N))

N = 67


In [ ]:
res = 0.0

for i in range(1, N+1): # [1, N+1) = [1, N]
    res += math.factorial(N) / (math.factorial(i) * math.factorial(N-i))

print("El número de combinaciones posibles = " + str(res))

El número de combinaciones posibles = 1.4757395258967638e+20


El número es excesivo así que no es viable testear la efectividad del algoritmo con todas y cada una de las combinaciones posible.

## Evaluación de variables de forma aislada

Para simplificar un poco el problema y tener una idea más o menos de cuál puede ser una buena combinación de variables, evaluaremos la efectividad de cada variable de forma aislada:

In [ ]:
res = []

feats = np.delete(X.columns, X.columns.size - 1)
for f in feats:
    scores = cross_val_score(GaussianNB(), X[[f]], y, cv=10)
    res.append((f, scores.mean()))

Ordenamos las variables por efectividad y las imprimimos:

In [ ]:
def get_score(elem):
    return elem[1]

res.sort(key=get_score, reverse=True)

for i in res:
    print(i)

('mean_aZ', 0.8476190476190476)
('median_aZ', 0.8238095238095238)
('q3_aZ', 0.8238095238095238)
('q1_aZ', 0.819047619047619)
('median_gY', 0.8095238095238095)
('q3_aX', 0.7904761904761903)
('min_gY', 0.738095238095238)
('iqr_gY', 0.7333333333333333)
('mean_aX', 0.7190476190476189)
('mean_gY', 0.7142857142857142)
('q1_gY', 0.7142857142857142)
('median_aX', 0.6952380952380952)
('max_aX', 0.6952380952380952)
('iqr_aX', 0.6952380952380952)
('stdev_gX', 0.6904761904761905)
('median_aY', 0.680952380952381)
('stdev_aX', 0.680952380952381)
('mean_gX', 0.6714285714285714)
('var_gX', 0.6714285714285714)
('stdev_gY', 0.6666666666666666)
('min_gX', 0.6666666666666666)
('var_aX', 0.6523809523809524)
('max_aZ', 0.6523809523809524)
('q1_gZ', 0.6523809523809524)
('mean_aY', 0.6476190476190476)
('var_aY', 0.6476190476190476)
('skew_aZ', 0.6476190476190475)
('q3_aY', 0.6476190476190475)
('q3_gY', 0.6428571428571429)
('max_gX', 0.6333333333333334)
('kurtosis_gY', 0.6333333333333333)
('var_gY', 0.62380952

Como primera aproximación está bien porque usando solo las X primeras variables conseguimos la misma tasa de acierto que usando todas las variables. Pero queremos seguir indagando para saber si existe un subconjunto de variables que no solo iguale la tasa de acierto del modelo con todas las variables, sino que incluso la mejore. Para ello necesitamos utilizar procedimientos más sofisticados.

## RFE con el algoritmo de Random Forest

El método RFE consiste en partir del conjunto total de features e ir eliminando de una en una las features menos relevantes hasta llegar a un subconjunto de tamaño *M*. Este método requiere el uso de un algoritmo de machine learning que vaya dejando rastro de la relevancia de cada feature a la hora de clasificar. Por ello vamos a usar RFE junto con el algoritmo de Random Forest.

Sklearn implementa ya una variante de RFE combinada con Cross Validation para encontrar el subconjunto de features óptimo. Utilizaremos esta variante para seleccionar las features:

In [ ]:
selector = RFECV(RandomForestClassifier(random_state=0), cv=10)
selector = selector.fit(X, y)

columns = X.columns.to_numpy()
feat_select_mask = selector.support_
selection = columns[feat_select_mask]
selection

array(['mean_aX', 'mean_aZ', 'median_aY', 'median_aZ', 'stdev_aX',
       'var_aY', 'min_aY', 'min_aZ', 'max_aY', 'kurtosis_aX', 'skew_aZ',
       'q1_aZ', 'q3_aX', 'q3_aZ', 'iqr_aX', 'iqr_aY', 'rms_a', 'mean_gX',
       'mean_gY', 'median_gX', 'median_gY', 'stdev_gX', 'stdev_gY',
       'var_gX', 'var_gY', 'min_gX', 'min_gY', 'max_gX', 'max_gZ',
       'kurtosis_gZ', 'skew_gX', 'q1_gY', 'q3_gX', 'iqr_gY', 'rms_g'],
      dtype=object)

En este punto ya tenemos los datos procesados cargados en las variable *X* e *y*, y además también tenemos el subconjunto óptimo de features a usar en la variable *selection*.

# Entrenando modelo de Machine Learning

Entrenamos el clasificador usando solamente las features seleccionadas

In [ ]:
X = X[selection]

#X = X[['mean_aY','mean_aZ','mean_gY','median_aY','median_aZ','median_gY','min_aZ','min_gY','max_aZ','q1_aY','q1_aZ','q1_gY','q3_aY','q3_aZ','q3_gY']]
#X = X.drop(['median_aX','median_aY','median_aZ','median_gX','median_gY','median_gZ','q1_aX','q1_aY','q1_aZ','q1_gX','q1_gY','q1_gZ','q3_aX','q3_aY','q3_aZ','q3_gX','q3_gY','q3_gZ','iqr_aX','iqr_aY','iqr_aZ','iqr_gX','iqr_gY','iqr_gZ','rms_a','rms_g'], axis=1)

model = GaussianNB().fit(X, y)

# Evaluando modelo

Mostramos la matriz de confusión

In [ ]:
y_pred = model.predict(X)

cm = pd.DataFrame(data=metrics.confusion_matrix(y, y_pred, labels=[0, 1, 2]), columns=['S_pred','O_pred','F_pred'], index=['S_real','O_real','F_real'])
cm

,S_pred,O_pred,F_pred
S_real,70,0,0
O_real,1,69,0
F_real,0,0,70


Utilizamos validación cruzada para medir la tasa de aciertos media con datos no conocidos

In [ ]:
scores = cross_val_score(GaussianNB(), X, y, cv=10)
print("%0.4f de aciertos con una desviación estándar de %0.4f" % (scores.mean(), scores.std()))

0.9524 de aciertos con una desviación estándar de 0.0563


# Exportando modelo a código C++

Convertimos el modelo a código C. Se generará la cabecera "trained_model.h" en la sección de archivos de este notebook.

In [ ]:
cmodel = emlearn.convert(model)
cmodel.save(file='trained_model.h')

'// !!! This file is generated by emlearn !!!\n\n    #include <eml_bayes.h>\n    \n\nEmlBayesSummary trained_model_summaries[105] = {\n        { EML_Q16_FROMFLOAT(-0.09760122448979591),EML_Q16_FROMFLOAT(0.2060260237006479),EML_Q16_FROMFLOAT(-2.279101515281723) },\n  { EML_Q16_FROMFLOAT(-0.7965936734693877),EML_Q16_FROMFLOAT(0.22775197678850365),EML_Q16_FROMFLOAT(-2.134464518837668) },\n  { EML_Q16_FROMFLOAT(-0.5856714285714286),EML_Q16_FROMFLOAT(0.22029123708690018),EML_Q16_FROMFLOAT(-2.182515987332521) },\n  { EML_Q16_FROMFLOAT(-0.8270214285714284),EML_Q16_FROMFLOAT(0.21512329735723254),EML_Q16_FROMFLOAT(-2.2167643211180965) },\n  { EML_Q16_FROMFLOAT(0.12393441269530088),EML_Q16_FROMFLOAT(0.19927348809757978),EML_Q16_FROMFLOAT(-2.3271783121282694) },\n  { EML_Q16_FROMFLOAT(0.12301259870845477),EML_Q16_FROMFLOAT(0.200027672539255),EML_Q16_FROMFLOAT(-2.321728493519976) },\n  { EML_Q16_FROMFLOAT(-1.438142857142857),EML_Q16_FROMFLOAT(0.3260308523564867),EML_Q16_FROMFLOAT(-1.61691960148740

A continuación se debe descargar la cabecera generada y colocarla en el código del proyecto (CronuSwim/src/)

# Generación de ficheros para tests unitarios


En este apartado se van a generar en la carpeta TFG/data/test/ todos los ficheros necesarios para que funcionen los tests unitarios correctamente. Una vez ejecutado todo, descargar la carpeta test y meterla directamente en el proyecto.

De esta forma evitamos tener los valores de prueba escritos directamente en el código fuente de los tests, mejorando mucho la mantenibilidad y haciendo posible que se puedan cambiar parámetros del software fácilmente sin romper los tests, únicamente generando los nuevos ficheros automáticamente.

## Ficheros event.csv copia

In [ ]:
for i in range(1, 32):
    if i <= 30:
        TEST_EVENT_FILE = 'real_event' + str(i) + '.csv'
    else:
        TEST_EVENT_FILE = 'event27.csv'

    src = FILE_PREFIX + TEST_EVENT_FILE
    target = TESTING_PATH + TEST_EVENT_FILE
    
    %cp -v "$src" "$target"

'/content/drive/MyDrive/US/4º_(III)/TFG/data/real_event1.csv' -> '/content/drive/MyDrive/US/4º_(III)/TFG/data/test/real_event1.csv'
'/content/drive/MyDrive/US/4º_(III)/TFG/data/real_event2.csv' -> '/content/drive/MyDrive/US/4º_(III)/TFG/data/test/real_event2.csv'
'/content/drive/MyDrive/US/4º_(III)/TFG/data/real_event3.csv' -> '/content/drive/MyDrive/US/4º_(III)/TFG/data/test/real_event3.csv'
'/content/drive/MyDrive/US/4º_(III)/TFG/data/real_event4.csv' -> '/content/drive/MyDrive/US/4º_(III)/TFG/data/test/real_event4.csv'
'/content/drive/MyDrive/US/4º_(III)/TFG/data/real_event5.csv' -> '/content/drive/MyDrive/US/4º_(III)/TFG/data/test/real_event5.csv'
'/content/drive/MyDrive/US/4º_(III)/TFG/data/real_event6.csv' -> '/content/drive/MyDrive/US/4º_(III)/TFG/data/test/real_event6.csv'
'/content/drive/MyDrive/US/4º_(III)/TFG/data/real_event7.csv' -> '/content/drive/MyDrive/US/4º_(III)/TFG/data/test/real_event7.csv'
'/content/drive/MyDrive/US/4º_(III)/TFG/data/real_event8.csv' -> '/content/d

## Ficheros de ventanas aisladas

En primer lugar necesitamos crear ficheros que contengan una única ventana y sepamos de antemano que tipo de ventana es. Por lo tanto se generarán ficheros con el nombre SX.csv que corresponderan a salidas, FX.csv que corresponderán a llegadas y OX.csv para otros tipos de ventanas, donde X es un número entero.

In [ ]:
def create_phase_file(section, filename_src, phase_class):
    if phase_class == 'S':
        global STARTS_COUNT
        STARTS_COUNT += 1
        filename_num = str(STARTS_COUNT)
    elif phase_class == 'F':
        global FINISH_COUNT
        FINISH_COUNT += 1
        filename_num = str(FINISH_COUNT)
    else:
        global OTHERS_COUNT
        OTHERS_COUNT += 1
        filename_num = str(OTHERS_COUNT)
    
    filename_target = TESTING_PATH + phase_class + filename_num + '.csv'
    
    event_df = pd.read_csv(filename_src)
    window = event_df[section[0]:section[1]]
    window.to_csv(filename_target, index=False)


# Divide un event.csv en S.csv, O.csv y F.csv
def split_in_phases_files(filename_src, sections):
    create_phase_file(sections[0], filename_src, 'S')
    create_phase_file(sections[1], filename_src, 'F')
    create_phase_file(random.choice(sections[2:]), filename_src, 'O')


# =============================== Main function =============================== #

def split_data():
    for filename in glob.glob(FILE_PREFIX + "event*.csv"):
        sections = get_sections(filename)
        split_in_phases_files(filename, sections)

    print("Window size: " + str(W_SIZE))

Llamamos a la función principal

In [ ]:
STARTS_COUNT = 0
OTHERS_COUNT = 0
FINISH_COUNT = 0

split_data()

Window size: 70


## Fichero con las estadísticas seleccionadas de una ventana en concreto

También necesitamos generar un fichero que contenga las estadísticas seleccionadas de una ventana en concreto que será la que se usará en los tests unitarios para realizar las pruebas.

In [ ]:
# =============================== Main function =============================== #

def create_statistic_from_file(filename_src):
    filename_target = "stats_" + filename_src
    section = (0, W_SIZE)

    event_df = pd.read_csv(TESTING_PATH + filename_src)
    timestamp = event_df['n'][section[0]:section[1]].mean()
    accel_stats = get_statistic_from_sensor('aX', 'aY', 'aZ', event_df, section)
    gyros_stats = get_statistic_from_sensor('gX', 'gY', 'gZ', event_df, section)
    stats = np.array(accel_stats + gyros_stats)
    relevant_stats = stats[feat_select_mask]
    str_relevant_stats = ','.join(str(s) for s in relevant_stats)

    with open(TESTING_PATH + filename_target, 'w') as file_target:
        stats_names = ','.join(selection)
        file_target.write('timestamp,' + stats_names + '\n') # header
        file_target.write(str(timestamp) + ',' + str_relevant_stats + '\n')

    print("Relevant features: ")
    print(str_relevant_stats)
    print("Timestamp: ")
    print(timestamp)

Llamamos a la función principal:

In [ ]:
create_statistic_from_file(TEST_EVENT_FILE)

Relevant features: 
-0.042571428571428545,-0.844757142857143,-0.551,-0.8440000000000001,0.008591642830717085,8.365489795918355e-05,-0.58,-0.857,-0.537,0.8826789660749395,-0.45971082883461356,-0.84875,-0.037000000000000005,-0.841,0.009999999999999995,0.010000000000000009,0.5832706143153532,-1.597314285714286,0.4220142857142857,-1.709,0.42700000000000005,0.7256668173452044,0.8561055257528064,0.5265923297959183,0.7329166712244891,-2.8689999999999998,-2.136,-0.061,0.8540000000000001,-0.4723143922086641,0.251614250886061,-0.04575,-0.9617499999999999,1.1295,1.359930688009678
Timestamp: 
113193.5
